In [242]:
import numpy as np
import non_rigid_registration as non
import normPts as normPts
import build_correspondence as build_c
import scipy.io
import deformation_transfer
data=scipy.io.loadmat('all_you_need.mat')

vs=data['VS']
fs=data['FS']
vt=data['VT']
ft=data['FT']
vs2=data['VS2']
fs2=data['FS2']
marker=data['marker']

vs_reg,vt_reg=non.non_rigid_registration(vs,fs,vt,ft,1,0.01,marker)
#print(vs_reg,vt_reg)
corres=build_c.build_correspondence(vs_reg,fs,vt_reg,ft,10,0.05)
scipy.io.savemat('corres')
I,C=deformation_transfer.deformation_transfer(vs,fs,vt,ft,vs2,fs2,corres)
print('Success')
#scipy.io.savemat('vs.mat',{'vertices':x,'triangle':ft})

normalize source and target vertices
Align Source to Target vertices
Building adjacency...
Solving Phase 1 optimization...
buiilding phase 1
Build phase 1 finished
VSP-finished
Start deformation transfer...
[array([], dtype=float64), array([], dtype=float64), array([], dtype=float64), array([], dtype=float64), array([], dtype=float64), array([], dtype=float64), array([], dtype=float64), array([], dtype=float64), array([], dtype=float64), array([], dtype=float64), array([], dtype=float64), array([], dtype=float64), array([], dtype=float64), array([], dtype=float64), array([], dtype=float64), array([], dtype=float64), array([], dtype=float64), array([], dtype=float64), array([], dtype=float64), array([], dtype=float64), array([], dtype=float64), array([], dtype=float64), array([], dtype=float64), array([], dtype=float64), array([], dtype=float64), array([], dtype=float64), array([], dtype=float64), array([], dtype=float64), array([], dtype=float64), array([], dtype=float64), array([], dt

IndexError: index 0 is out of bounds for axis 0 with size 0

In [4]:
import v4_normal as v4
import build_elementary_cell as build_e
import numpy as np
import scipy.sparse.linalg
import scipy.sparse

In [243]:
corres_data=scipy.io.loadmat('corres.mat')

In [244]:

corres=corres_data['corres']

In [247]:
corres.shape

(31620, 1)

In [90]:
corres_data=scipy.io.loadmat('corres.mat')
corres=corres_data['corres']

In [91]:
len(corres)

31620

In [252]:
corres[1006][0][0]

array([11907, 11908, 12183, 12459, 13017, 13298, 13581, 13866],
      dtype=uint16)

In [253]:
n_corres = 0
n_non_corres=0
for i in range(len(corres)):
    if len(corres[i][0]):
        n_corres += len(corres[i][0][0])
    else:
        n_non_corres+=1

In [256]:
n_corres

157350

In [257]:
lenfs=len(fs)
lenft=len(ft)
SD=[]
ts,ns,vs4,fs4=v4.v4_normal(vs,fs)
ts2,ns2,vs42,fs42=v4.v4_normal(vs2,fs2)
tt,nt,vt4,ft4=v4.v4_normal(vt,ft)

for i in range(lenfs):
    SD.append(np.matmul(ts2[i],np.linalg.inv(ts[i])))#矩阵除法

E=build_e.build_elementary_cell(tt,lenft)
E=np.around(E,4)

n_corres = 0
n_non_corres=0

#for i in range(len(corres)):
#    if len(corres[i]):
#       n_corres += len(corres[i])
#   else:
#        n_non_corres+=1

for i in range(len(corres)):
    if len(corres[i][0]):
        n_corres += len(corres[i][0][0])
    else:
        n_non_corres+=1


I=np.zeros([9*(n_corres+n_non_corres)*4,3])
C=np.zeros([9*(n_corres+n_non_corres),1])

print('Transfer deformation')

offset=0
offset2=0


for i in range(lenft):
    if len(corres[i][0]):
        lencor=len(corres[i][0][0])
        cor=corres[i][0][0]
    else:
        lencor=0
        cor=corres[i][0]

    U=ft4[i]
    if lencor:
        for j in range(lencor):
            for k in range(3):
                row=np.matlib.repmat(np.array([1,2,3])+offset+j*3*3+k*3,4,1)
                col1=np.matlib.repmat((U-1)*3+k+1,3,1)
                col1=np.transpose(col1)
                val1=np.transpose(E[i])

                a=np.matlib.reshape(row,12,1)
                b=np.matlib.reshape(col1,12,1)
                c=np.matlib.reshape(val1,12,1)
                I[np.arange(0,12)+offset2+j*3*3*4+k*3*4]=np.transpose(np.vstack((a,b,c)))
               

            C[np.arange(0,9)+offset+j*9,0]=np.matlib.reshape(np.transpose(SD[int(cor[j])-1]),9,1)

        offset=offset+3*3*lencor
        offset2=offset2+3*3*lencor*4
    else:
        for k in range(3):
            row=np.matlib.repmat(np.array([1,2,3])+offset+k*3,4,1)
            col1=np.matlib.repmat((U-1)*3+k+1,3,1)
            col1=np.transpose(col1)
            val1=np.transpose(E[i])
            a=np.matlib.reshape(row,12,1)
            b=np.matlib.reshape(col1,12,1)
            c=np.matlib.reshape(val1,12,1)

            I[np.arange(0,12)+offset2+k*3*4]=np.transpose(np.vstack((a,b,c)))
            

        C[np.arange(0,9)+offset,0]=np.matlib.reshape(np.eye(3),9,1)
        offset=offset+3*3
        offset2=offset2+3*3*4
#return I

print('Finish transfer.')

Transfer deformation
Finish transfer.


In [259]:
C

array([[1.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [1.]])

In [212]:
i=208
j=12
k=2
offset=1872
offset2=7488

In [194]:
row=np.matlib.repmat(np.array([1,2,3])+offset+j*3*3+k*3,4,1)

In [199]:
col1=np.matlib.repmat((U-1)*3+k+1,3,1)
col1=np.transpose(col1)

In [236]:
temp=E[7284]

In [239]:
np.around(E,4)

array([[[  0.0878,  -0.8748,  -0.0891,   0.8761],
        [ -2.4363,   4.0888,  -1.5049,  -0.1476],
        [ -3.1299,   2.9849,  -0.314 ,   0.459 ]],

       [[ -0.905 ,  -1.7291,   1.9018,   0.7324],
        [  0.9948,   1.7549,  -2.9366,   0.1869],
        [ -0.7989,   1.4331,  -1.289 ,   0.6548]],

       [[ -1.5513,   1.8984,  -0.6605,   0.3134],
        [ -0.6868,  -0.7948,   0.7958,   0.6859],
        [ -0.0652,  -0.0757,  -0.5159,   0.6568]],

       ...,

       [[  0.8826,  -1.1119,   1.1992,  -0.97  ],
        [ 13.5791,  -0.8751, -12.7965,   0.0925],
        [ -6.2239,  -4.4345,  10.4335,   0.2249]],

       [[  0.69  ,  -0.475 ,   0.7549,  -0.97  ],
        [  2.894 ,  -7.6692,   4.6826,   0.0925],
        [ -2.6602,   1.1055,   1.3298,   0.2249]],

       [[ -0.0864,   0.0435,   1.0129,  -0.97  ],
        [ -9.6421,  -6.8039,  16.3536,   0.0925],
        [ -0.8532,   2.9855,  -2.3573,   0.2249]]])

In [203]:
row=np.matlib.repmat(np.array([1,2,3])+offset+j*3*3+k*3,4,1)
col1=np.matlib.repmat((U-1)*3+k+1,3,1)
col1=np.transpose(col1)
val1=np.transpose(E[i])

a=np.matlib.reshape(row,12,1)
b=np.matlib.reshape(col1,12,1)
c=np.matlib.reshape(val1,12,1)
I[np.arange(0,12)+offset2+j*3*3*4+k*3*4]=np.transpose(np.vstack((a,b,c)))



In [260]:
scipy.io.savemat('./IandC2.mat',{'I':I,'C':C})

In [144]:
cor

array([11895, 11897, 11898, 11899, 11900, 11901, 12173, 12174, 12448,
       12449, 12726, 12727, 13004], dtype=uint16)

In [145]:
np.matlib.reshape(np.transpose(SD[int(cor[j-1])-1]),9,1)

array([ 0.97137521,  0.02439196,  0.2432717 , -0.30725987,  1.27482523,
        0.10574608, -0.12978055, -0.39915718,  1.00113641])

In [102]:
import python2matlab
python2matlab.matlabsolver2()

AttributeError: module 'python2matlab' has no attribute 'matlabsolver2'

In [120]:
from scipy.sparse import csr_matrix
from scipy import array
import matlab
import matlab.engine
eng = matlab.engine.start_matlab()
x = eng.solver1()


In [121]:
np.array(x)

array([[ 4.05813434],
       [-9.15406042],
       [-8.68412872],
       ...,
       [-7.39942446],
       [ 3.91304046],
       [-3.88916727]])

In [165]:
te=scipy.io.loadmat('IandC2.mat')

In [166]:
tee=te['C']

In [167]:
tee[10000]

array([0.09820617])

In [108]:
temp=np.array(x)

In [109]:
temp

array([[ 4.05813434],
       [-9.15406042],
       [-8.68412872],
       ...,
       [-7.39942446],
       [ 3.91304046],
       [-3.88916727]])

In [83]:
M=scipy.sparse.csr_matrix((I[:,2],(I[:,0]-1,I[:,1]-1)),(int(np.max(I[:,0])),int(np.max(I[:,1]))))

In [87]:
temp_M=scipy.sparse.csr_matrix.dot(np.transpose(M),M)

In [96]:
temp_C=scipy.sparse.csr_matrix.dot(np.transpose(M),C)

In [97]:
temp_C

array([[ 1.5154486 ],
       [ 0.10595374],
       [ 0.66988029],
       ...,
       [-0.96996987],
       [ 0.09250114],
       [ 0.22494886]])

In [80]:
temp_M[0,0]

51.625975700076026

In [62]:
scipy.io.savemat('vs.mat',{'vertices':x,'triangle':ft})

In [63]:
import scipy.io
data = scipy.io.loadmat('./vs.mat') 
vertices = data['vertices']
triangles = data['triangle']


# In[2]:


#vertices = vertices['target_vertices']


# In[3]:


#triangles = triangles['triangles']


# In[4]:


obj_file = './result.obj'
with open(obj_file, 'w') as f:
    for v in range(0,vertices.shape[0]):
        f.write('v %0.2f %0.2f %0.2f %0.2f %0.2f %0.2f\n' % (vertices[v,0],vertices[v,1],vertices[v,2],0,0,0))

    for t in range(0,triangles.shape[0]):
        f.write('f {} {} {}\n'.format(*triangles[t,:]+1))

print('Calculated the isosurface, save at obj file:',obj_file)

Calculated the isosurface, save at obj file: ./result.obj


In [262]:
import scipy.io
data = scipy.io.loadmat('./vs.mat') 
vertices = data['vertices']
triangles = data['triangle']
n_vertices=data['n_vertices']


# In[2]:


#vertices = vertices['target_vertices']


# In[3]:


#triangles = triangles['triangles']


# In[4]:


obj_file = './result.obj'
with open(obj_file, 'w') as f:
    for v in range(0,vertices.shape[0]):
        f.write('v %0.2f %0.2f %0.2f\n' % (vertices[v,0],vertices[v,1],vertices[v,2]))
    for n in range(0,vertices.shape[0]):
        f.write('n %0.2f %0.2f %0.2f\n' % (n_vertices[n,0],n_vertices[n,1],n_vertices[n,2]))

    for t in range(0,triangles.shape[0]):
        f.write('f {} {} {}\n'.format(*triangles[t,:]+1))

print('Calculated the isosurface, save at obj file:',obj_file)

Calculated the isosurface, save at obj file: ./result.obj


In [263]:
vertices

array([[  4.24525898, -14.41319752,   0.17980565],
       [  3.81360374, -15.18745264,   0.74817276],
       [  4.11564548, -15.17488083,   0.41710401],
       ...,
       [  5.58628116,  -2.44252615,   0.71980858],
       [  0.89700524,  -3.20660787,   9.95086079],
       [  4.44721474, -14.34030199,  -0.17749493]])